In [1]:
import pandas as pd
tmp = pd.read_csv("data/run2/data.csv")

In [2]:
tmp["Signs"].mean(), tmp["Lights"].mean(), tmp["Vehicles"].mean()
#[0.602, 1.087, 1.203]

(0.0003187562499999907, 0.0011545166666666433, 0.07146388124999997)

In [3]:
tmp["Unlabeled"].median(),tmp["Vehicles"].median(), tmp["Signs"].median(),tmp["Lights"].median(),  tmp["All"].median()

(0.9300697916666667,
 0.06845729166666659,
 1.875e-05,
 0.00047187499999995,
 0.0699302083333333)

In [2]:
signs = tmp["Signs"].median()
lights = tmp["Lights"].median()
vehicles = tmp["Vehicles"].median()
test = tmp["Signs"] + tmp["Lights"] + tmp["Vehicles"]
a = test.median()
a/vehicles, a/signs, a/lights

(1.0239707051482794, 21251.999999999953, 234.82872928178816)

In [10]:
test = tmp["Signs"] + tmp["Lights"] + tmp["Vehicles"]
test.median()

0.06972812499999995

In [8]:
med_frq = [0.382900, 0.452448, 0.637584, 0.377464, 0.585595,
           0.479574, 0.781544, 0.982534, 1.017466, 0.624581,
           2.589096, 0.980794, 0.920340, 0.667984, 1.172291,
           0.862240, 0.921714, 2.154782, 1.187832, 1.178115,
           1.848545, 1.428922, 2.849658, 0.771605, 1.656668,
           4.483506, 2.209922, 1.120280, 2.790182, 0.706519,
           3.994768, 2.220004, 0.972934, 1.481525, 5.342475,
           0.750738, 4.040773]

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

label_colours = [(0, 0, 0),
                 # 0=background
                 (148, 65, 137), (255, 116, 69), (86, 156, 137),
                 (202, 179, 158), (155, 99, 235), (161, 107, 108),
                 (133, 160, 103), (76, 152, 126), (84, 62, 35),
                 (44, 80, 130), (31, 184, 157), (101, 144, 77),
                 (23, 197, 62), (141, 168, 145), (142, 151, 136),
                 (115, 201, 77), (100, 216, 255), (57, 156, 36),
                 (88, 108, 129), (105, 129, 112), (42, 137, 126),
                 (155, 108, 249), (166, 148, 143), (81, 91, 87),
                 (100, 124, 51), (73, 131, 121), (157, 210, 220),
                 (134, 181, 60), (221, 223, 147), (123, 108, 131),
                 (161, 66, 179), (163, 221, 160), (31, 146, 98),
                 (99, 121, 30), (49, 89, 240), (116, 108, 9),
                 (161, 176, 169), (80, 29, 135), (177, 105, 197),
                 (139, 110, 246)]
len(label_colours), len(med_frq)

(41, 37)

In [1]:
import carla
import random
import queue
import numpy as np
import cv2

client = carla.Client('localhost', 2000)
world  = client.get_world()
bp_lib = world.get_blueprint_library()

In [2]:
spawn_points = world.get_map().get_spawn_points()
vehicle_bp =bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# for i in range(50):
#     vehicle_bp = random.choice(bp_lib.filter('vehicle'))
#     npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
#     if npc:
#         npc.set_autopilot(True)

# spawn camera
camera_r = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(z=2))
camera_rgb = world.spawn_actor(camera_r, camera_init_trans, attach_to=vehicle)

camera_d = bp_lib.find('sensor.camera.depth')
camera_init_trans = carla.Transform(carla.Location(z=2))
camera_depth = world.spawn_actor(camera_d, camera_init_trans, attach_to=vehicle)

camera_s = bp_lib.find('sensor.camera.semantic_segmentation')
camera_init_trans = carla.Transform(carla.Location(z=2))
camera_semantic = world.spawn_actor(camera_s, camera_init_trans, attach_to=vehicle)

vehicle.set_autopilot(True)

# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 5
world.apply_settings(settings)

# Create a queue to store and retrieve the sensor data
rgb_queue = queue.Queue()
camera_rgb.listen(rgb_queue.put)
depth_queue = queue.Queue()
camera_depth.listen(depth_queue.put)
semantic_queue = queue.Queue()
camera_semantic.listen(semantic_queue.put)

In [37]:
import carla
import random
import queue
import numpy as np
import cv2

client = carla.Client('localhost', 2000)
world  = client.get_world()
test = []
for i in world.get_actors():
    test.append(i.type_id)
list(set(test))

['spectator',
 'traffic.speed_limit.30',
 'traffic.speed_limit.60',
 'static.prop.mesh',
 'traffic.speed_limit.90',
 'traffic.traffic_light']

In [ ]:
'traffic.traffic_light',
'vehicle'

In [ ]:
world.tick()
actor_list = []
for i in world.get_actors().filter('*vehicle*'):
    actor_list.append(i)
for i in world.get_actors().filter('*traffic*'):
    actor_list.append(i)
vehicles = 0
trafficsigns = 0
trafficlight = 0
for curr in actor_list:
    if curr.id != vehicle.id:
        bb = curr.bounding_box
        dist = curr.get_transform().location.distance(vehicle.get_transform().location)
        print(dist)
        if dist < 50:
            print("test")
            forward_vec = vehicle.get_transform().get_forward_vector()
            ray = curr.get_transform().location - vehicle.get_transform().location

            if forward_vec.dot(ray) > 1:

In [3]:
image_w = camera_r.get_attribute("image_size_x").as_int()
image_h = camera_r.get_attribute("image_size_y").as_int()
fov = camera_r.get_attribute("fov").as_float
image_w, image_h, fov

(800,
 600,
 <bound method as_float of <carla.libcarla.ActorAttribute object at 0x0000024C52E85CF0>>)

In [4]:
image_w = camera_d.get_attribute("image_size_x").as_int()
image_h = camera_d.get_attribute("image_size_y").as_int()
fov = camera_d.get_attribute("fov").as_float
image_w, image_h, fov

(800,
 600,
 <bound method as_float of <carla.libcarla.ActorAttribute object at 0x0000024C52E85DB0>>)

In [14]:
world.tick()
image = rgb_queue.get()
img_rgb = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
cv2.imwrite("out/test_rgb.png", img_rgb)

image = depth_queue.get()
image.convert(carla.ColorConverter.LogarithmicDepth)
img_depth = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
cv2.imwrite("out/test_depth.png", img_depth)

image = semantic_queue.get()
image.convert(carla.ColorConverter.CityScapesPalette)
img_semantic = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
cv2.imwrite("out/test_semantic.png", img_semantic)

True

In [4]:
world.tick()
imaged = depth_queue.get()
imager = rgb_queue.get()

In [9]:
array = np.reshape(np.copy(imaged.raw_data), (imaged.height, imaged.width, 4))
normalized_depth = np.dot(array[:, :, :3], [65536.0, 256.0, 1.0])
normalized_depth /= 16777215.0  # (256.0 * 256.0 * 256.0 - 1.0)
normalized_depth.shape

(600, 800)

In [5]:
img_semantic = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
img_semantic.shape

(600, 800, 4)

In [8]:
img_semantic[:,:,2]

array([[13, 13, 13, ...,  9,  9,  9],
       [13, 13, 13, ...,  9,  9,  9],
       [13, 13, 13, ...,  9,  9,  9],
       ...,
       [ 7,  7,  7, ...,  8,  8,  8],
       [ 7,  7,  7, ...,  8,  8,  8],
       [ 7,  7,  7, ...,  8,  8,  8]], dtype=uint8)